# Проект по SQL

## Описание проекта

## Подключение к БД

Подготовим подключение к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': '', # имя пользователя
'pwd': '', # пароль
'host': '',
'port': , # порт подключения
'db': ''} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Изучение данных

Изучим таблицы с данными

In [2]:
query_books = ''' SELECT *
            FROM books
            LIMIT 5
        '''

In [3]:
pd.io.sql.read_sql(query_books, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query_authors = ''' SELECT *
            FROM authors
            LIMIT 5
        '''

In [5]:
pd.io.sql.read_sql(query_authors, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query_publishers = ''' SELECT *
            FROM publishers
            LIMIT 5
        '''

In [7]:
pd.io.sql.read_sql(query_publishers, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
query_ratings = ''' SELECT *
            FROM ratings
            LIMIT 5
        '''

In [9]:
pd.io.sql.read_sql(query_ratings, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [10]:
query_reviews = ''' SELECT *
            FROM reviews
            LIMIT 5
        '''

In [11]:
pd.io.sql.read_sql(query_reviews, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Задание 1

Посчитайте, сколько книг вышло после 1 января 2000 года;

In [12]:
query_1 = ''' SELECT COUNT(DISTINCT book_id)
                FROM books
                WHERE EXTRACT(YEAR FROM CAST (publication_date AS timestamp)) >= 2000
        '''

In [13]:
pd.io.sql.read_sql(query_1, con = engine)

,count
0,821


Всего с 1 января 2000 года вышла 821 книга

## Задание 2

Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [14]:
query_2 = ''' SELECT DISTINCT b.book_id AS book,
              b.title,
              COUNT(DISTINCT rev.review_id) AS reviews_qty,
              ROUND(AVG(rat.rating), 2) AS rating_avg     
              FROM books AS b
              LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
              LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
              GROUP BY book
              ORDER BY reviews_qty DESC,
              rating_avg DESC
        '''

In [15]:
pd.io.sql.read_sql(query_2, con = engine)

,book,title,reviews_qty,rating_avg
0,948,Twilight (Twilight #1),7,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,656,The Book Thief,6,4.26
4,734,The Glass Castle,6,4.21
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.00
996,221,Essential Tales and Poems,0,4.00
997,387,Leonardo's Notebooks,0,4.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


Больше всего обзоров на первую часть серии книг "Сумерки". По ней написано 7 рецензий, средняя оценка 3,66. На втором месте по количеству обзоров и средней оценке третья книга серии о Гарри Поттере - "Гарри Поттер и Узник Азкабана". По ней написано 6 резенций, средний рейтинг 4,41. Тройку замыкает вторая книга из той же серии - "Гарри Поттер и Тайная комната". По ней написано 6 резенций, средний рейтинг 4,29.

## Задание 3

Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [16]:
query_3 = ''' 
              SELECT publisher_id,
              publisher
              FROM publishers
              WHERE publisher_id IN (SELECT publisher_id
                              FROM books
                              WHERE book_id IN (SELECT book_id
                                        FROM books
                                        WHERE num_pages > 50)
                              GROUP BY publisher_id
                              ORDER BY COUNT(DISTINCT book_id) DESC
                              LIMIT 1); 
        '''

In [17]:
pd.io.sql.read_sql(query_3, con = engine)

,publisher_id,publisher
0,212,Penguin Books


Penguin Books выпустило наибольшее число книг толще 50 страниц

## Задание 4

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [18]:
query_4 = ''' SELECT author_id,
              author
              FROM authors
              WHERE author_id IN
                          (SELECT b.author_id
                          FROM books AS b
                          JOIN ratings AS r ON b.book_id = r.book_id
                          WHERE b.book_id IN
                                            (SELECT book_id
                                            FROM ratings
                                            GROUP BY book_id
                                            HAVING COUNT(rating_id) > 50)
                          GROUP BY b.author_id
                          ORDER BY AVG(r.rating) DESC
                          LIMIT 1)
        '''

In [19]:
pd.io.sql.read_sql(query_4, con = engine)

,author_id,author
0,236,J.K. Rowling/Mary GrandPré


J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книг

## Задание 5

Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [20]:
query_5 = ''' 
            WITH a AS(
            SELECT username,
            COUNT(DISTINCT review_id) AS review_qty
            FROM reviews
            WHERE username IN(
                            SELECT username
                            FROM ratings
                            GROUP BY username
                            HAVING COUNT(rating_id) > 50)
            GROUP BY username)

            SELECT ROUND(AVG(review_qty))
            FROM a
        '''
pd.io.sql.read_sql(query_5, con = engine)

,round
0,24.0


Пользователи, которые поставили больше 50 оценок, в среднем оставили 24 обзора

## Вывод

База данных состоит из 5 таблиц: books - содержит данные о книгах, authors - данные об авторах, ratings - данные о пользовательских оценках книг, reviews - данные о пользовательских обзорах, publishers - данные об издательствах.

Всего в базе данных содержится информация о 340 издательствах, 636 авторах и 1000 книгах (821 из них вышла после 2000 года). Всего 160 пользователей оставили свою оценку/обзор на книгу.

- Суммарно по количеству обзоров и средней оценке лидирует третья книга серии о Гарри Поттере - "Гарри Поттер и Узник Азкабана". По ней написано 6 обзоров, средний рейтинг составил 4,41.

- В среднем самые активные пользователи (те, кто поставили более 50 оценок) оставляли 24 обзора на книги.

- J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книг.

- Говоря об издательстве, здесь можно выделить Penguin Books. Они выпустили больше всего книг (без учета брошюр, т.е. книги более чем с 50 страницами).